In [1]:
!pip install kaggle-environments -U

In [2]:
from kaggle_environments import make

Loading environment football failed: No module named 'gfootball'


In [3]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

In [4]:
# snippet to find the closest city tile to a position
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

In [28]:
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
    
    # Fuel only gets used up at night so we need enough to last the nights
    
    #Default= build new cities unless not enough fuel...
    new_city = True
    
    #Copy resource tiles 
    resource_tiles_copy=resource_tiles.copy()
    
    for city in player.cities.values():
        req_fuel = 90 * city.get_light_upkeep() # There are 90 nights total

        if city.fuel < req_fuel:
            # let's not build a new one yet
            new_city = False
            
        # Do stuff with our citytiles
        for tile in city.citytiles:
            if tile.can_act():
                
                # If we have fewer units than cities create a unit
                if len(player.units) < sum([len(city.citytiles) for city in player.cities.values()]):
                    action = tile.build_worker()
                    actions.append(action)
                
                # Otherwise do research
                else:
                    action = tile.research()
                    actions.append(action)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            
            # Find the closest city tile and its distance from the unit
            closest_city_tile = find_closest_city_tile(unit.pos, player)
            d = unit.pos.distance_to(closest_city_tile.pos)
            
            if observation["step"] % 40 >= 26: #  FIX THIS LATER. Make it go home properly.
                action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                actions.append(action)
                continue
                
            
            if (unit.can_build(game_state.map) and new_city and d > 0) or closest_city_tile is None:
                    action = unit.build_city()
                    actions.append(action)
                
            
            # we want to mine only if there is space left in the worker's cargo
            elif unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                
                closest_resource_tile = find_closest_resources(unit.pos, player, resource_tiles_copy)
                
                #Dont let agents have the same closest resource (dont compete and collide, hopefully)
                i= resource_tiles_copy.index(closest_resource_tile)
                
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_resource_tile.pos))
                    
                    
                    actions.append(action)
                del resource_tiles_copy[i]
            else:
                # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                if closest_city_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                    actions.append(action)
                    
    
    
    return actions

In [31]:
env = make("lux_ai_2021", configuration={"seed": 69420, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])

[WARN] (match_egl3Dw0IGL6t) - turn 103; Unit u_3 collided when trying to move s to (6, 2)
[WARN] (match_egl3Dw0IGL6t) - turn 132; Unit u_1 collided when trying to move s to (6, 2)
[WARN] (match_egl3Dw0IGL6t) - turn 138; Unit u_3 collided when trying to move s to (8, 2)
[WARN] (match_egl3Dw0IGL6t) - turn 142; Unit u_1 collided when trying to move e to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - turn 144; Unit u_3 collided when trying to move e to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - Agent 0 tried to build unit on tile (4, 1) but unit cap reached. Build more CityTiles!; turn 220; cmd: bw 4 1
[WARN] (match_egl3Dw0IGL6t) - turn 221; Unit u_1 collided when trying to move n to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - turn 221; Unit u_4 collided when trying to move e to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - turn 222; Unit u_1 collided when trying to move n to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - turn 222; Unit u_4 collided when trying to move e to (8, 1)
[WARN] (match_egl3Dw0IGL6t) - turn 223; Unit u_1 c

In [32]:
env.render(mode="ipython", width=900, height=600)


In [8]:
%%writefile agent.py
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
    
    # Fuel only gets used up at night so we need enough to last the nights
    new_city = True
    
    for city in player.cities.values():
        req_fuel = 90 * city.get_light_upkeep() # There are 90 nights total

        if city.fuel < req_fuel:
            # let's not build a new one yet
            new_city = False
            
        # Do stuff with our citytiles
        for tile in city.citytiles:
            if tile.can_act():
                
                # If we have fewer units than cities create a unit
                if len(player.units) < sum([len(city.citytiles) for city in player.cities.values()]):
                    action = tile.build_worker()
                    actions.append(action)
                
                # Otherwise do research
                else:
                    action = tile.research()
                    actions.append(action)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            
            # Find the closest city tile and its distance from the unit
            closest_city_tile = find_closest_city_tile(unit.pos, player)
            d = unit.pos.distance_to(closest_city_tile.pos)
            
            if observation["step"] % 40 >= 26: #  FIX THIS LATER. Make it go home properly.
                action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                actions.append(action)
                continue
                
            
            if (unit.can_build(game_state.map) and new_city and d==1) or closest_city_tile is None:
                action = unit.build_city()
                actions.append(action)
                
            
            # we want to mine only if there is space left in the worker's cargo
            elif unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile = find_closest_resources(unit.pos, player, resource_tiles)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_resource_tile.pos))
                    actions.append(action)
            else:
                # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                if closest_city_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                    actions.append(action)
                    
    
    
    return actions

Overwriting agent.py


## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [9]:
!tar -czf submission.tar.gz *

tar: submission.tar.gz: Can't add archive to itself
